In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
!pip install fer opencv-python-headless imageio[ffmpeg]

# Install ffmpeg
!apt-get update
!apt-get install -y ffmpeg


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,771 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,104 kB]
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:14 http://securi

In [ ]:
import os
import cv2
from fer import FER
import json

# Set the FFmpeg path
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

# Initialize the FER detector
detector = FER(mtcnn=True)

# Load the video file from Google Drive
video_path = "/content/drive/MyDrive/Video_Gen/Bahubali_2_scene.mp4"
video = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not video.isOpened():
    print("Error: Could not open video.")
    exit()

# Process the video frame by frame
while True:
    ret, frame = video.read()

    if not ret:
        break

    # Detect emotions in the frame
    result = detector.detect_emotions(frame)

    # Print the result
    print(json.dumps(result, indent=4))

    # Display the frame with emotions (optional)
    for face in result:
        (x, y, w, h) = face["box"]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        emotion = max(face["emotions"], key=face["emotions"].get)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Display the frame in Google Colab (optional)
    from google.colab.patches import cv2_imshow
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close display window
video.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from fer import FER
from collections import Counter

# Initialize FER emotion detector
emotion_detector = FER()

def analyze_video_emotions(video_path):
    print(f"Opening video file: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return None, None

    frame_count = 0
    emotion_sums = {
        'angry': 0,
        'disgust': 0,
        'fear': 0,
        'happy': 0,
        'sad': 0,
        'surprise': 0,
        'neutral': 0
    }
    all_emotions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Analyze the emotions in the frame
        emotions = emotion_detector.detect_emotions(frame)
        if emotions:
            dominant_emotion = max(emotions[0]['emotions'], key=emotions[0]['emotions'].get)
            all_emotions.append(dominant_emotion)
            for emotion, score in emotions[0]['emotions'].items():
                emotion_sums[emotion] += score
            frame_count += 1

    cap.release()

    if frame_count == 0:
        print("No frames were processed.")
        return None, None

    # Calculate average emotions
    average_emotions = {emotion: score / frame_count for emotion, score in emotion_sums.items()}

    # Determine the predominant emotion
    emotion_counts = Counter(all_emotions)
    predominant_emotion = emotion_counts.most_common(1)[0][0] if all_emotions else None

    print(f"Processed {frame_count} frames.")
    return predominant_emotion, average_emotions

# Analyze the video
video_path = '/content/drive/MyDrive/Video_Gen/Bahubali_2_scene.mp4'
result = analyze_video_emotions(video_path)

if result:
    predominant_emotion, average_emotions = result
    if predominant_emotion:
        print(f"Predominant emotion: {predominant_emotion}")
    else:
        print("No predominant emotion detected.")
    if average_emotions:
        print("Average emotions:")
        for emotion, score in average_emotions.items():
            print(f"{emotion}: {score:.2f}")
else:
    print("No frames were processed in the video.")


Opening video file: /content/drive/MyDrive/Video_Gen/Bahubali_2_scene.mp4
Processed 5307 frames.
Predominant emotion: neutral
Average emotions:
angry: 0.22
disgust: 0.00
fear: 0.11
happy: 0.08
sad: 0.16
surprise: 0.06
neutral: 0.37
